In [1]:
import pandas as pd
import re
import datetime as dt
import numpy as np

Esercizio 1.

Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.

In [2]:
ll=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\loans_lenders.csv')

In [6]:
def dividi(stringa):
    return re.split(',',stringa)
ll['lista']=ll.apply(lambda row: dividi(row['lenders']), axis=1)

In [7]:
lenders=[nome for lista in ll['lista'] for nome in lista]


In [8]:
ll['lunghezza']=[len(x) for x in ll['lista']]

In [9]:
listone=[]
i=0
for i in range(0,len(ll)):
    listone.append([ll['loan_id'][i]]*ll['lunghezza'][i])

In [10]:
loan_id=[y for x in listone for y in x]

In [11]:
lo_le=pd.DataFrame({'loan_id':loan_id, 'lenders':lenders})

In [16]:
lo_le.head(10)

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499
5,483693,karlheinz4543
6,483693,jerrydb
7,483693,paula8951
8,483693,gmct
9,483693,amra9383


In [14]:
lo_le.tail(10)

,loan_id,lenders
28293921,1206425,rich6705
28293922,1206425,sergiiy9766
28293923,1206425,angela7509
28293924,1206425,barbara5610
28293925,1206425,david9667
28293926,1206425,trogdorfamily7622
28293927,1206425,danny6470
28293928,1206425,don6118
28293929,1206486,alan5175
28293930,1206486,amy38101311


Esercizio 2.

For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.


In [3]:
l=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\loans.csv')

In [18]:
l['planned_expiration_time']=pd.to_datetime(l['planned_expiration_time'])
l['disburse_time']=pd.to_datetime(l['disburse_time'])
l['duration']=abs(l['planned_expiration_time']-l['disburse_time'])
l[['loan_id','duration']].head(10)


,loan_id,duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
5,659605,35 days 18:30:07
6,660240,31 days 23:50:11
7,661601,46 days 01:50:03
8,661960,33 days 16:10:03
9,662117,68 days 05:40:02


In [19]:
l[['loan_id','duration']].tail(10)

,loan_id,duration
1419597,984735,63 days 06:20:02
1419598,985764,46 days 02:40:05
1419599,986756,44 days 12:20:04
1419600,986789,40 days 11:50:04
1419601,987606,42 days 08:30:11
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02
1419606,989240,61 days 12:50:06


Esercizio 3. 

Find the lenders that have funded at least twice.


In [22]:
lenders_twice=lo_le.groupby('lenders').count()
lenders_twice2=lenders_twice[lenders_twice['loan_id']>1]
lenders_twice2=lenders_twice2.reset_index()
lenders_twice2[['lenders']].head(10)

,lenders
0,000
1,00000
2,0002
3,0101craign0101
4,0132575
5,0154884
6,0161130
7,0169713
8,0185429
9,0206338


In [21]:
lenders_twice2[['lenders']].tail(10)

,lenders
1005101,zuzanna7983
1005102,zuzanna9818
1005103,zvi
1005104,zx147
1005105,zyrah8525
1005106,zyrorl
1005107,zzaman
1005108,zzanita
1005109,zzmcfate
1005110,zzrvmf8538


Esercizio 4.

For each country, compute how many loans have involved that country as borrowers.


In [24]:
paesi=l.groupby('country_name').size()
paesi

country_name
Afghanistan                               2337
Albania                                   3075
Armenia                                  13952
Azerbaijan                               10172
Belize                                     218
Benin                                     5946
Bhutan                                       2
Bolivia                                  25250
Bosnia and Herzegovina                     608
Botswana                                     1
Brazil                                     482
Bulgaria                                   296
Burkina Faso                              3489
Burundi                                   1727
Cambodia                                 79701
Cameroon                                  5085
Canada                                       1
Chad                                        61
Chile                                      892
China                                      134
Colombia                                 33675


Esercizio 5.

For each country, compute the overall amount of money borrowed.


In [26]:
l.groupby('country_name')['loan_amount'].sum()

country_name
Afghanistan                              1967950.0
Albania                                  4307350.0
Armenia                                 22950475.0
Azerbaijan                              14784625.0
Belize                                    150175.0
Benin                                    3865825.0
Bhutan                                     20000.0
Bolivia                                 44226725.0
Bosnia and Herzegovina                    477250.0
Botswana                                    8000.0
Brazil                                   1192325.0
Bulgaria                                  375300.0
Burkina Faso                             4085200.0
Burundi                                  5233450.0
Cambodia                                51613525.0
Cameroon                                 2250800.0
Canada                                     50000.0
Chad                                       20075.0
Chile                                    2595925.0
China             

Esercizio 6.

Like the previous point, but expressed as a percentage of the overall amount lent.


In [27]:
stati=l.groupby('country_name')[['loan_amount']].sum()
totale=sum(stati['loan_amount'])
stati['loan_percentage']=(stati['loan_amount']/totale)*100
stati=stati['loan_percentage']

In [29]:
stati

country_name
Afghanistan                             0.166573
Albania                                 0.364586
Armenia                                 1.942589
Azerbaijan                              1.251410
Belize                                  0.012711
Benin                                   0.327214
Bhutan                                  0.001693
Bolivia                                 3.743468
Bosnia and Herzegovina                  0.040396
Botswana                                0.000677
Brazil                                  0.100922
Bulgaria                                0.031766
Burkina Faso                            0.345782
Burundi                                 0.442973
Cambodia                                4.368706
Cameroon                                0.190514
Canada                                  0.004232
Chad                                    0.001699
Chile                                   0.219726
China                                   0.032209
Colombi

Esercizio 7.

Like the three previous points, but split for each year (with respect to disburse time).


In [32]:
l['year']=pd.to_datetime(l['disburse_time']).dt.year

In [33]:
paesi2=l.groupby(['year','country_name']).size()

In [35]:
tot=l.groupby('year')[['loan_amount']].sum()

In [40]:
stati2=l.groupby(['year','country_name'])[['loan_amount']].sum()
stati2=stati2.reset_index() 
nuovo=pd.merge(stati2, tot, on='year')
nuovo['percentage']=(nuovo['loan_amount_x']/nuovo['loan_amount_y'])*100
nuovo=nuovo[['year','country_name','percentage']]
nuovo.head(10)

,year,country_name,percentage
0,2005.0,Bulgaria,1.798736
1,2005.0,Cambodia,1.579971
2,2005.0,Ecuador,4.861449
3,2005.0,Gaza,4.861449
4,2005.0,Honduras,32.377248
5,2005.0,Kenya,20.758386
6,2005.0,Nicaragua,3.208556
7,2005.0,Senegal,3.135634
8,2005.0,Tanzania,2.236266
9,2005.0,Uganda,25.182304


In [38]:
nuovo.tail(10)

,year,country_name,percentage
738,2018.0,Peru,1.787834
739,2018.0,Philippines,0.030302
740,2018.0,Rwanda,2.707002
741,2018.0,Senegal,0.401505
742,2018.0,Sierra Leone,3.143860
743,2018.0,Tanzania,1.444408
744,2018.0,Thailand,2.676700
745,2018.0,Uganda,2.196914
746,2018.0,Uruguay,0.808060
747,2018.0,Zambia,1.411581


Esercizio 8.

For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.


In [42]:
l['single_fund']=l['funded_amount']/l['num_lenders_total']

In [43]:
new=pd.merge(l, ll, on='loan_id')[['loan_id','lenders','single_fund']]

In [44]:
new2=pd.merge(new,lo_le,on='loan_id')[['loan_id', 'single_fund', 'lenders_y']]

In [45]:
money_lent=new2.groupby('lenders_y')[['single_fund']].sum()

In [46]:
money_lent.head(10)

,single_fund
lenders_y,
000,1485.309656
00000,1249.127691
0002,2201.180463
00mike00,38.461538
0101craign0101,2424.088932
0132575,113.587695
0154884,122.077677
0161130,52.739726
0169713,90.689866


In [47]:
money_lent.tail(10)

,single_fund
lenders_y,
zyra9641,26.886792
zyrah8525,166.696429
zyrorl,92.599588
zzaba,25.000000
zzaman,385.071453
zzanita,62.500000
zzcyna7269,44.285714
zzinnia,32.758621
zzmcfate,1889.702578


Esercizio 9.

For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.


In [4]:
lend=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\lenders.csv')

In [48]:
vuoto=lend[lend['country_code'].isnull()==True]

In [49]:
pieno=lend[lend['country_code'].isnull()==False]

In [50]:
conteggio=pieno.groupby(['country_code']).size()

In [51]:
conteggio=conteggio.reset_index()

In [52]:
stati=[x for x in conteggio['country_code']]

In [53]:
volte=[x for x in conteggio[0]]

In [54]:
listone2=[]
for i in range(0,len(stati)):
    listone2.append([stati[i]]*volte[i])

In [55]:
end=[y for x in listone2 for y in x]

In [108]:
import random
vect=[]
for i in range(0,len(vuoto)):
    vect.append(random.sample(end,1)[0])

In [109]:
vuoto['country_code']=vect

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [110]:
new_lenders=pieno.append(vuoto)

In [111]:
new_lenders.rename(columns={'permanent_name': 'lenders_y'}, inplace=True)


In [112]:
prestatori=pd.merge(new2, new_lenders, on='lenders_y')[['loan_id', 'lenders_y', 'country_code', 'single_fund']]

In [113]:
somma_prestati=prestatori.groupby('country_code')[['single_fund']].sum()

In [114]:
somma_ricevuti=l.groupby('country_code')[['funded_amount']].sum()

In [115]:
entrambi=pd.merge(somma_prestati, somma_ricevuti, on = 'country_code', how='left')

In [116]:
prest=[]
for i in range(0, len(entrambi)):
    if pd.isna(entrambi.iloc[i]['funded_amount'])==True:
        prest.append(0)
    else:
        prest.append(entrambi.iloc[i]['funded_amount'])
entrambi['funded_amount']=prest       

In [117]:
entrambi['differenza']=entrambi['single_fund']-entrambi['funded_amount']

In [118]:
entrambi.head(10)

,single_fund,funded_amount,differenza
country_code,,,
AD,189.617940,0.0,1.896179e+02
AE,102682.653115,0.0,1.026827e+05
AF,5973.946964,1964650.0,-1.958676e+06
AL,1647.847903,3992475.0,-3.990827e+06
AM,827.215070,20284675.0,-2.028385e+07
AN,1348.349728,0.0,1.348350e+03
AO,1512.022744,0.0,1.512023e+03
AQ,2047.085994,0.0,2.047086e+03
AR,53240.394093,0.0,5.324039e+04


In [119]:
entrambi.tail(10)

,single_fund,funded_amount,differenza
country_code,,,
VN,18925.859258,23463825.0,-2.344490e+07
VU,15125.652111,9250.0,5.875652e+03
WF,168.847752,0.0,1.688478e+02
WS,130.892857,11508250.0,-1.150812e+07
XK,795.176981,2862325.0,-2.861530e+06
YE,8118.369053,3340025.0,-3.331907e+06
YT,52.118848,0.0,5.211885e+01
ZA,28340.096963,1006525.0,-9.781849e+05
ZM,828.390744,1913250.0,-1.912422e+06


Esercizio 10.

Which country has the highest ratio between the difference computed at the previous point and the population?


In [120]:
country_stats=pd.read_csv('C:\\Users\\symo9\\Desktop\\FoCS\\progetto\\country_stats.csv')

In [121]:
iniziale=pd.merge(country_stats, entrambi, on='country_code')[['country_code', 'population_below_poverty_line', 'population', 'differenza']]

In [122]:
primo_rapporto=iniziale
primo_rapporto['first_ratio']=primo_rapporto['differenza']/primo_rapporto['population']

In [123]:
fine_primo=primo_rapporto[['country_code', 'first_ratio']]

In [124]:
vincitore=fine_primo.iloc[fine_primo['first_ratio'].idxmax()]

In [125]:
vincitore

country_code          LU
first_ratio     0.227434
Name: 152, dtype: object

Esercizio 11.

Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?


In [126]:
secondo_rapporto=iniziale
secondo_rapporto['second_ratio']=secondo_rapporto['differenza']/(100-secondo_rapporto['population_below_poverty_line'])

In [127]:
fine_secondo=secondo_rapporto[['country_code', 'second_ratio']]

In [128]:
vincitore2=fine_secondo.iloc[fine_secondo['second_ratio'].idxmax()]

In [129]:
vincitore2

country_code         US
second_ratio    74710.8
Name: 6, dtype: object

Esercizio 12.

For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.


In [132]:
l['year2']=pd.to_datetime(l['planned_expiration_time']).dt.year

In [133]:
l1=l[(l['year2']-l['year'])>=1]
l1=l1.reset_index()
l2=l[(l['year2']-l['year'])==0]
l2=l2.reset_index()
ammonti=[x for x in l2['loan_amount']]
years=[x for x in l2['year']]
c=pd.DataFrame({'loan':ammonti, 'anni':years}) 

In [134]:
lista_finale_pesi=[]
lista_finale_anni=[]

In [135]:
for i in range(0,len(l1)):
    diff=l1['year2'][i]-l1['year'][i]
    k=0
    lista2=[]
    for k in range(0,int(diff)+1):
        lista2.append(l1['year'][i]+k)
        if k==int(diff) and int(diff)>1:
            lista1=[x for x in range(0,int(diff)+1)]
            for j in range(1, k):
                lista1[j]=365
        else:
            lista1=[x for x in range(0,2)]
    lista1[0]=365-(pd.to_datetime(l1['disburse_time'][i])).dayofyear
    lista1[-1]=(pd.to_datetime(l1['planned_expiration_time'][i])).dayofyear
    pesi=[x*l1['loan_amount'][i]/sum(lista1) for x in lista1]
    for h in range(0,len(pesi)):
        lista_finale_pesi.append(pesi[h])
        lista_finale_anni.append(lista2[h])    

In [136]:
b=pd.DataFrame({'loan':lista_finale_pesi, 'anni':lista_finale_anni})

In [137]:
finale=c.append(b)

In [138]:
finale.groupby('anni')[['loan']].sum()

,loan
anni,
2011.0,6.299119e+05
2012.0,1.131230e+08
2013.0,1.250890e+08
2014.0,1.531511e+08
2015.0,1.597600e+08
2016.0,1.587487e+08
2017.0,1.721008e+08
2018.0,5.705071e+06
